# PROMISE 你可能真的还没用明白（续集进阶版）

本文引用自[PROMISE 你可能真的还没用明白（续集进阶版）](https://www.freesion.com/article/72201330875/)

文章分析：
- 文章首先给出了Promise同步、异步的划分，解释了针对Promise这个场景，微任务机制是如何生效的。
- 探索了几个有趣的场景，对于原理探究，这样的实验是可行的；对于实际应用，除了初级中的代码，其他实验代码的方式都是不可取的。

## PROMISE 哪些 API 涉及了微任务？

Promise 中只有涉及到状态变更后才需要被执行的回调才算是微任务，比如说 `then`、` catch` 、`finally` ，其他所有的代码执行都是宏任务（同步执行）。

![](https://www.freesion.com/images/848/d8125ea51bae39a4a3d75eed17a11748.png)

上图中蓝色为同步执行，黄色为异步执行（丢到微任务队列中）。

## 这些微任务何时被加入微任务队列？

这个问题我们根据 ecma 规范来看：

- 如果此时 Promise 状态为 pending，那么成功或失败的回调会分别被加入至 `[[PromiseFulfillReactions]] ` 和 `[[PromiseRejectReactions]] ` 中。如果你看过手写 Promise 的代码的话，应该能发现有两个数组存储这些回调函数。

- 如果此时 Promise 状态为非 pending 时，回调会成为 Promise Jobs，也就是微任务。

## 同一个 THEN，不同的微任务执行

### 初级

> 可能出现的场景，但如果逻辑是连续执行的，可以转化为三个同级的`then`调用；一般情况是没必要调用`Promise.resolve()`触发一个Promise链的

In [3]:
Promise.resolve()

  .then(() => {

    console.log("then1");

    Promise.resolve().then(() => {

      console.log("then1-1");

    });

  })

  .then(() => {

    console.log("then2");

  });

then1
then1-1
then2


以上代码大家应该都能得出正确的答案：`then1 → then1-1 → then2`。

虽然 `then`  是同步执行，并且状态也已经变更。但这并不代表每次遇到 `then` 时我们都需要把它的回调丢入微任务队列中，而是等待 `then` 的回调执行完毕后再根据情况执行对应操作。

基于此，我们可以得出第一个结论：链式调用中，只有前一个 `then ` 的回调执行完毕后，跟着的 `then ` 中的回调才会被加入至微任务队列。

## 中级

大家都知道了 `Promise resolve ` 后，跟着的 `then ` 中的回调会马上进入微任务队列。

那么以下代码你认为的输出会是什么？

> 这种给同一个Promise多次添加then回调的写法，直接避免

In [4]:
var p = Promise.resolve();

p.then(() => {

  console.log("then1");

  Promise.resolve().then(() => {

    console.log("then1-1");

  });

}).then(() => {

  console.log("then1-2");

});

p.then(() => {

  console.log("then2");

});

then1
then2
then1-1
then1-2


按照一开始的认知我们不难得出 `then2 ` 会在 `then1-1 ` 后输出，但是实际情况却是相反的。

基于此我们得出第二个结论：每个链式调用的开端会首先依次进入微任务队列。

接下来我们换个写法：

> 同样，这种给同一个Promise多次添加then回调的写法，直接避免

In [1]:
var p = Promise.resolve().then(() => {

  console.log("then1");

  Promise.resolve().then(() => {

    console.log("then1-1");

  });

}).then(() => {

  console.log("then2");

});

p.then(() => {

  console.log("then3");

});

then1
then1-1
then2
then3


上述代码其实有个陷阱，`then ` 每次都会返回一个新的 Promise，此时的 `p ` 已经不是 `Promise.resolve() ` 生成的，而是最后一个 `then ` 生成的，因此 `then3 ` 应该是在 `then2 ` 后打印出来的。

顺便我们也可以把之前得出的结论优化为：同一个 Promise 的每个链式调用的开端会首先依次进入微任务队列。

## 高级

以下大家可以猜猜 `then1-2 ` 会在何时打印出来？

> 

In [2]:
Promise.resolve()

  .then(() => {

    console.log("then1");

    Promise.resolve()

      .then(() => {

        console.log("then1-1");

        return 1;

      })

      .then(() => {

        console.log("then1-2");

      });

  })

  .then(() => {

    console.log("then2");

  })

  .then(() => {

    console.log("then3");

  })

  .then(() => {

    console.log("then4");

  });

then1
then1-1
then2
then1-2
then3
then4


这题肯定是简单的，记住第一个结论就能得出答案，以下是解析：

- 第一次 `resolve ` 后第一个 `then ` 的回调进入微任务队列并执行，打印 `then1 `

- 第二次 `resolve ` 后内部第一个 `then ` 的回调进入微任务队列，此时外部第一个 `then ` 的回调全部执行完毕，需要将外部的第二个 `then ` 回调也插入微任务队列。

- 执行微任务，打印 `then1-1 ` 和 `then2 `，然后分别再将之后 `then ` 中的回调插入微任务队列

- 执行微任务，打印 `then1-2 ` 和 `then3 ` ，之后的内容就不一一说明了

接下来我们把 `return 1 ` 修改一下，结果可就大不相同啦：

In [3]:
Promise.resolve()

  .then(() => {

    console.log("then1");

    Promise.resolve()

      .then(() => {

        console.log("then1-1");

        return Promise.resolve();

      })

      .then(() => {

        console.log("then1-2");

      });

  })

  .then(() => {

    console.log("then2");

  })

  .then(() => {

    console.log("then3");

  })

  .then(() => {

    console.log("then4");

  });

then1
then1-1
then2
then3
then4
then1-2


当我们 `return Promise.resolve() ` 时，你猜猜 `then1-2 ` 会何时打印了？

答案是最后一个才被打印出来。

为什么在 `then ` 中分别 `return ` 不同的东西，微任务的执行顺序竟有如此大的变化？以下是笔者的解析。

PS：`then `** 返回一个新的 Promise，并且会用这个 Promise 去 `resolve ` 返回值，这个概念需要大家先了解一下。**

### 根据 PROMISE A+ 规范

根据规范 2.3.2[2]，如果 `resolve ` 了一个 Promise，需要为其加上一个 `then ` 并 `resolve `。

```js
if (x instanceof MyPromise) {

  if (x.currentState === PENDING) {

  } else {

    x.then(resolve, reject);

  }

  return;

}
```

上述代码节选自手写 Promise 实现。

那么根据 A+ 规范来说，如果我们在 `then ` 中返回了 `Promise.resolve ` 的话会多入队一次微任务，但是这个结论还是与实际不符的，因此我们还需要寻找其他权威的文档。

### 根据 ECMA - 262 规范

根据规范 25.6.1.3.2[3]，当 `Promise resolve ` 了一个 Promise 时，会产生一个 NewPromiseResolveThenableJob，这是属于 Promise Jobs 中的一种，也就是微任务。

```

This Job uses the supplied thenable and its then method to resolve the given promise. This process must take place as a Job to ensure that the evaluation of the then method occurs after evaluation of any surrounding code has completed.

```

并且该 Jobs 还会调用一次 `then ` 函数来 `resolve Promise `，这也就又生成了一次微任务。

这就是为什么会触发两次微任务的来源。